In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
def get_cars(url, starting_page=1, page_lim=5):
    global df_train
    rows = []
    for i in range(starting_page, page_lim+1):
        param = {'page':i}
        response = requests.get(url, param)
        cars = BeautifulSoup(response.text, 'html.parser')
        list_of_cars = cars.find_all('div',
                                     class_='ListingItemSequential-module__container')
        car_links = list(
            map(
                lambda x: x.find(
                    'div',
                    class_='ListingItemSequential-module__summaryLeft'
                ).a.get('href'), list_of_cars
            )
        )
    
        for link in car_links:
            time.sleep(0.4)
            response = requests.get(link)
            response.encoding = 'utf-8'
            car = BeautifulSoup(response.text, 'html.parser')
            table = car.find(class_='CardInfo')
            model_link = car.find('a', class_='CardCatalogLink').get('href')
            specific_car = {'production_date':[table.find('li', class_='CardInfoRow_year').a.text],
                            'mileage':[table.find('li', class_='CardInfoRow_kmAge').text],
                            'carconfig':[table.find('li', class_='CardInfoRow_bodytype').a.text],
                            'colour':[table.find('li', class_='CardInfoRow_color').a.text],
                            'transmission':[table.find('li', class_='CardInfoRow_transmission').text],
                            'drive_unit':[table.find('li', class_='CardInfoRow_drive').text],
                            'wheel':[table.find('li', class_='CardInfoRow_wheel').text],
                            'condition':[table.find('li', class_='CardInfoRow_state').text],
                            'licence':[table.find('li', class_='CardInfoRow_pts').text],
                            'customs':[table.find('li', class_='CardInfoRow CardInfoRow_customs').text],
                            'price':[car.find('span', class_='OfferPriceCaption__price').text]}
            model = requests.get(model_link)
            model.encoding = 'utf-8'
            model_info = BeautifulSoup(model.text, 'html.parser')
            tables = model_info.find_all('dl', class_='list-values')
            specific_car['brand'] = model_info.find('div', class_='search-form-v2-mmm__breadcrumbs'
                                                        ).find_all('a')[0].text
            specific_car['model_name'] = model_info.find('div', class_='search-form-v2-mmm__breadcrumbs'
                                                        ).find_all('a')[1].text
            specific_car['model_date'] = model_info.find('div', class_='search-form-v2-mmm__breadcrumbs'
                                                        ).find_all('a')[2].text
            for listed in tables:
                time.sleep(0.1)
                k = len(listed.find_all('dt'))
                for i in range(0, k):
                    name = listed.find_all('dt')[i].text
                    value = listed.find_all('dd')[i].text
                    specific_car[name] = value
                
                
                
            try:
                specific_car['engine'] = table.find('li', class_='CardInfoRow_engine').find('div').text
            except:
                specific_car['engine'] = None
            if str(type(table.find('li', class_='CardInfoRow_ownersCount'))) == "<class 'bs4.element.Tag'>":
                specific_car['owners'] = table.find('li', class_='CardInfoRow_ownersCount').text
            else:
                specific_car['owners'] = None    
            if str(type(table.find('li', class_='CardInfoRow_owningTime'))) == "<class 'bs4.element.Tag'>":
                specific_car['age'] = table.find('li', class_='CardInfoRow_owningTime').text
            else:
                specific_car['age'] = None
            df = pd.DataFrame.from_dict(specific_car)
            rows.append(df)
    df_train = pd.concat(rows)
    return df_train

In [3]:
df_1 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 1, 5)
df_2 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 6, 10)
df_3 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 11, 15)
df_4 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 16, 20)
df_5 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 21, 25)
df_6 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 26, 30)
df_7 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 31, 35)
df_8 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 36, 40)
df_9 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 41, 45)
df_10 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 46, 50)
df_11 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 51, 55)
df_12 = get_cars('https://auto.ru/cars/used/?year_to=2020&output_type=table', 56, 60)
df_train = pd.concat([df_1, df_2, df_3, df_4,
                      df_5, df_6, df_7, df_8,
                      df_9, df_10, df_11, df_12])

ChunkedEncodingError: ('Connection broken: IncompleteRead(2918 bytes read, 7322 more expected)', IncompleteRead(2918 bytes read, 7322 more expected))

In [ ]:
df_train.head()

In [ ]:
df_train = df_train.loc[~df_train.engine.isna()]

In [ ]:
df_train = df_train.reset_index().drop('index', axis = 1)

In [ ]:
df_train['mileage'] = df_train.mileage.apply(lambda x:x.replace('Пробег', ''))
df_train['mileage'] = df_train.mileage.apply(lambda x:x.replace('км', ''))
df_train['mileage'] = df_train.mileage.apply(lambda x:int(x.replace(u'\xa0', '')))

In [ ]:
df_train['engine'] = df_train.engine.apply(lambda x: x.split('/'))
df_train['engine_displacement'] = df_train.engine.apply(lambda x: (x[0]))
df_train['engine_displacement'] = df_train.engine_displacement.apply(lambda x: (x.replace('л', 'LTR')))
df_train['engine_displacement'] = df_train.engine_displacement.apply(lambda x: (x.replace('кВт', 'kW')))
df_train['power'] = df_train.engine.apply(lambda x: int(x[1].replace(u'\xa0л.с.', '')))
df_train['fuel_type'] = df_train.engine.apply(lambda x: x[2])
df_train = df_train.drop('engine', axis=1)

In [ ]:
df_train.transmission = df_train.transmission.apply(lambda x: x.replace('Коробка',''))
df_train.wheel = df_train.wheel.apply(lambda x: x.replace('Руль',''))
df_train.condition = df_train.condition.apply(lambda x: x.replace('Состояние',''))
df_train.owners = df_train.owners.apply(lambda x: x.replace('Владельцы',''))
df_train.licence = df_train.licence.apply(lambda x: x.replace('ПТС',''))
df_train.customs = df_train.customs.apply(lambda x: x.replace('Таможня',''))
df_train.drive_unit = df_train.drive_unit.apply(lambda x: x.replace('Привод',''))

In [ ]:
df_train['price'] = df_train.price.apply(lambda x: x.replace('₽', ''))
df_train['price'] = df_train.price.apply(lambda x: int(x.replace(u'\xa0', '')))

In [ ]:
not_empty = df_train.loc[(df_train.age.isna()==False)].index
df_train.loc[not_empty, 'age'] = df_train.iloc[not_empty].age.apply((lambda x: x.replace('Владение','')))

In [ ]:
df_train['number_of_doors'] = df_train['Количество дверей']
df_train = df_train.drop('Количество дверей', axis=1)

In [ ]:
df_train = df_train[
    (df_train['condition']=='Не требует ремонта') &
    (df_train['customs']=='Растаможен')
]
df_train = df_train.drop(['condition', 'customs'], axis=1)
df_train.head()

In [ ]:
columns_to_keep = ['production_date', 'mileage', 'model_date',
                   'carconfig', 'colour', 'transmission',
                   'drive_unit', 'wheel', 'owners',
                   'licence', 'brand', 'model_name',
                   'age', 'engine_displacement',
                   'power', 'fuel_type', 'number_of_doors',
                   'price']

In [ ]:
df_train = df_train[columns_to_keep]
df_train['test'] = 0

In [ ]:
df_train

In [ ]:
df_train.to_csv('train.csv', index=True)